In [ ]:
import csv
import os
from dotenv import load_dotenv
import pandas as pd
import json
from datetime import datetime
import numpy as np

load_dotenv()

reg_filepath = os.environ['STAT_BY_REG_PATHFILE']

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(reg_filepath, sep=',', engine="c")

In [ ]:
mask_2023 = df.year == 2023
df.loc[mask_2023, 'first_day_of_week'] = df.loc[mask_2023, 'first_day_of_week'].str.replace('2023-', '2024-')


In [ ]:
goals = {
    'Île-de-France':339978,
    'Centre-Val de Loire': 77778,
    'Bourgogne-Franche-Comté':  80000,
    'Normandie':  102222,
    'Hauts-de-France':  204444,
    'Grand Est':  151111,
    'Pays de la Loire':120000,
    'Bretagne':  105556,
    'Nouvelle-Aquitaine': 177778,
    'Occitanie':200000,
    'Auvergne-Rhône-Alpes' :239132,
    'Provence-Alpes-Côte d\'Azur': 150000,
}


In [ ]:
# Conversion de la colonne first_day_of_week en datetime
df['first_day_of_week'] = pd.to_datetime(df['first_day_of_week'])

In [ ]:
mask_after_jun = df['first_day_of_week'] > datetime(2024, 6, 1)
df = df[mask_after_jun]

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

def plot_cumulative_sums_to_pdf(df, output_pdf):
    with PdfPages(output_pdf) as pdf:
        # Boucle pour chaque région
        for region in df['libelle'].unique():
            # Filtrer les données par région
            region_data = df[df['libelle'] == region]
            
            # Filtrer les données par année 2023 et 2024
            region_data_2023 = region_data[region_data['year'] == 2023]
            region_data_2024 = region_data[region_data['year'] == 2024]
            
            # Calcul des sommes cumulées
            region_data_2023 = region_data_2023.sort_values('first_day_of_week')
            region_data_2023['cumulative_sum'] = region_data_2023['sum'].cumsum()
            
            region_data_2024 = region_data_2024.sort_values('first_day_of_week')
            region_data_2024['cumulative_sum'] = region_data_2024['sum'].cumsum()
            
            # Tracer les courbes
            plt.figure(figsize=(10, 6))
            plt.plot(region_data_2023['first_day_of_week'], region_data_2023['cumulative_sum'], label='2023', marker='o')
            plt.plot(region_data_2024['first_day_of_week'], region_data_2024['cumulative_sum'], label='2024', marker='o')
            
            # Personnalisation du graphique
            plt.title(f'Évolution cumulée des inscrits pour la région {region}')
            plt.xlabel('Semaine')
            plt.ylabel("Somme cumulée du nombre d'inscrits")
            plt.legend()
            plt.grid(True)

            # Ajouter une ligne horizontale pour l'objectif (goal)
            if region in goals:
                goal_value = goals[region]
                goal_reached_value = region_data_2024['sum'].sum()
                goal_percentage_reached_vaue = round(region_data_2024['sum'].sum()*100/goal_value)

                plt.axhline(y=goal_value, color='red', linestyle='--')
                # Dessiner le texte de l'objectif sur le graphique
                # On place le texte légèrement à droite du graphique pour éviter l'axe
                plt.text(region_data_2024['first_day_of_week'].min(), 
                    goal_value + goal_value*0.02, 
                    f"Objectif final: {goal_value:,}".replace(',', ' '), 
                        color='red', fontsize=12, va='center')
                
                plt.axhline(y=goal_reached_value, linestyle='--', color='black')
                plt.text(region_data_2024['first_day_of_week'].min(), 
                    goal_reached_value + 400, 
                    f"Inscrits : {goal_reached_value}, ({goal_percentage_reached_vaue}% de l'objectif atteint)".replace(',', ' '), color='black', fontsize=12)
            
            plt.gca().get_yaxis().set_major_formatter(FuncFormatter(plain_number_formatter))
            # plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True, nbins=30))

            # Affichage du graphique
            plt.xticks(rotation=45)
            plt.tight_layout()
            # Enregistrer la figure dans le fichier PDF
            pdf.savefig()  
            # Affichage des graphs
            plt.show()
            plt.close()  

output_pdf_path = './cumulative_sums_by_region.pdf'
plot_cumulative_sums_to_pdf(df, output_pdf_path)

In [ ]:
# Draw cumulative graph total

from matplotlib.ticker import FuncFormatter, MaxNLocator

# Conversion de la colonne first_day_of_week en datetime
df['first_day_of_week'] = pd.to_datetime(df['first_day_of_week'])


# Fonction pour formater les nombres avec des zéros
def plain_number_formatter(x, pos):
    return f'{int(x):,}'.replace(',', ' ')

# Fonction pour tracer l'évolution cumulée des sommes par région
def plot_cumulative_sums(df):
    # Filtrer les données par année 2023 et 2024
    region_data_2023 = df[df['year'] == 2023]
    region_data_2024 = df[df['year'] == 2024]
    
    # Calcul des sommes cumulées
    region_data_2023 = region_data_2023.sort_values('first_day_of_week')
    region_data_2023['cumulative_sum'] = region_data_2023['sum'].cumsum()
    
    region_data_2024 = region_data_2024.sort_values('first_day_of_week')
    region_data_2024['cumulative_sum'] = region_data_2024['sum'].cumsum()
    
    # Tracer les courbes
    plt.figure(figsize=(10, 6))
    plt.plot(region_data_2023['first_day_of_week'], region_data_2023['cumulative_sum'], label='2023', marker='o')
    plt.plot(region_data_2024['first_day_of_week'], region_data_2024['cumulative_sum'], label='2024', marker='o')
    
    # Personnalisation du graphique
    plt.title(f'Évolution cumulée des sommes des inscriptions au pass')
    plt.xlabel('Date')
    plt.ylabel('Somme cumulée')
    plt.legend()
    plt.grid(True)
    
    plt.gca().get_yaxis().set_major_formatter(FuncFormatter(plain_number_formatter))
    # plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True, nbins=30))

    # Affichage du graphique
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Appel de la fonction pour tracer les graphiques
plot_cumulative_sums(df)